In [ ]:
# MCP Server & Client Implementation Using Fastmcp
# Architecture
# LLM Host (MCP Client) <—— JSON-RPC ——> fastmcp Server <——> CSV Dataset

# We'll:

# Launch a server exposing two tools (summarize, query).
# Use an MCP Client to discover and call those tools.
# Simulate a host (agent) deciding which tool to call.
# Start the MCP Server
# In a separate terminal, run:

# python mcp_server_fastmcp.py
# This starts an MCP-compliant server exposing summarize and query tools.

In [ ]:
%pip install fastmcp pandas openai json
# uvicorn --upgrade --no-cache-dir


In [ ]:
%pip install json

In [ ]:
# Connect via MCP Client
# Once the server is running, connect using the MCP client to list tools and invoke them.


from fastmcp import Client
from fastmcp.client import PythonStdioTransport
import asyncio

async def main():
    # Create a client using the PythonStdioTransport
    # Create a loop to run the client

    transport = "http://127.0.0.1:8765/mcp"
    
    # client = Client("http://127.0.0.1:8765")
    # transport = "http://127.0.0.1:8000/mcp"
    #transport = "ws://127.0.0.1:8765"

    async with Client(transport) as client:
        tools = await client.list_tools()
        print("Tools:", [t.name for t in tools])

    await main()

In [ ]:
# Simulate a Simple Host Agent
# Let's simulate a rule-based 'AI agent' that decides whether to use summarize or query based on user text.

def decide_tool(text: str):
    text = text.lower()
    if "summarize" in text or "overview" in text:
        return "summarize", {}
    if "west" in text:
        return "query", {"expr": "region == 'West' and sales > 1500"}
    return "summarize", {}

async def run_agent(user_input, client):
    tool, params = decide_tool(user_input) #"summarize", {}
    #tool = "summarize"
    #params = {}
    print(f"Agent decided to use '{tool}'")

    # API for fastmcp 2.12.5
    result = await client.call_tool(tool, params)
    # result = await client.call_tool("query", params)

    print("Result:", result, "\n")

In [ ]:
# create a connection to your MCP server
transport = "http://127.0.0.1:8765/mcp"
 
async with Client(transport) as client:
        await run_agent("Give me a summary of the dataset", client)

In [19]:
# create a connection to your MCP server

import json
transport = "http://127.0.0.1:8765/mcp"

async with Client(transport) as client:
     result = await run_agent("Show West region sales > 1500", client)
     

# 1. Read the parsed JSON (this is the important part)
data = result["rows"]  # Assuming the result has a "rows" key with the data

# 2. Print in JSON format
print(json.dumps(data, indent=2))

# 3. Sum total sales
total_sales = sum(row["sales"] for row in data["rows"])

print("Total sales =", total_sales)


Agent decided to use 'query'
Result: CallToolResult(content=[TextContent(type='text', text='{"count":2,"rows":[{"order_id":107,"region":"West","sales":2000,"quarter":"Q3","rep":"Gale"},{"order_id":110,"region":"West","sales":2200,"quarter":"Q4","rep":"Jules"}]}', annotations=None, meta=None)], structured_content={'count': 2, 'rows': [{'order_id': 107, 'region': 'West', 'sales': 2000, 'quarter': 'Q3', 'rep': 'Gale'}, {'order_id': 110, 'region': 'West', 'sales': 2200, 'quarter': 'Q4', 'rep': 'Jules'}]}, meta=None, data={'count': 2, 'rows': [{'order_id': 107, 'region': 'West', 'sales': 2000, 'quarter': 'Q3', 'rep': 'Gale'}, {'order_id': 110, 'region': 'West', 'sales': 2200, 'quarter': 'Q4', 'rep': 'Jules'}]}, is_error=False) 



TypeError: 'NoneType' object is not subscriptable

In [ ]:

# Architecture Recap
# +--------------------+      JSON-RPC (MCP)      +--------------------+
# |  Host / AI Agent   |  <-------------------->  | fastmcp MCP Server |
# | (LLM or Router)    |                        | (summarize, query)  |
# +--------------------+                         +--------------------+